In [7]:
import pandas as pd
import os
import glob

In [8]:
all_metadata_path = "mimic_all.csv"
all_metadata = pd.read_csv(all_metadata_path)

In [9]:
def generate_image_paths(row, index, total_rows):
    base_path = "/home/DATA/mimic-cxr/2.0.0/files_jpg_512/files"
    subject_id = str(row["subject_id"])
    study_id = str(row["study_id"])
    ref_id = str(row.get("ref_id", ""))

    def get_correct_image(subject_id, study_id):
        matching_row = all_metadata[
            (all_metadata["subject_id"] == int(subject_id)) &
            (all_metadata["study_id"] == int(study_id))
        ]
        if not matching_row.empty:
            return matching_row.iloc[0]["dicom_id"]
        return None
    
    correct_image = get_correct_image(subject_id, study_id)
    if correct_image:
        image_path = os.path.relpath(
            os.path.join(base_path, f"p{subject_id[:2]}/p{subject_id}/s{study_id}/{correct_image}.jpg"), base_path
        )
        if os.path.exists(os.path.join(base_path, image_path)):
            return image_path
    return ""

In [10]:
from tqdm import tqdm

# Now i wanto to generate the image paths for all the rows
all_metadata["image_path"] = ""
for index, row in tqdm(all_metadata.iterrows(), total=len(all_metadata)):
    all_metadata.at[index, "image_path"] = generate_image_paths(row, index, len(all_metadata))

all_metadata.to_csv("mimic_all_with_image_paths.csv", index=False)

  0%|          | 0/227827 [00:00<?, ?it/s]

100%|██████████| 227827/227827 [01:48<00:00, 2104.69it/s]
